In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import requests
import pandas as pd
import re, logging



In [4]:
url = 'https://astore-alanar.com/products/kf20'

In [63]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#options.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)

Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 1.1.4322)


/tmp/ipykernel_437317/4049218499.py:7: DeprecationWarning: use options instead of firefox_options
  driver = webdriver.Firefox(firefox_options=options)


In [44]:
urls = pd.read_excel('astore_url_update.xlsx')
list_urls = []
for index, row in urls.iterrows():
    list_urls.append(
        {
            'url': row['url'],
            'cat1': row['cat1'],
            'cat2': row['cat2'],
            'cat3': row['cat3'],
        }
    )


In [ ]:
colors_bois = driver.find_elements(By.XPATH, '//ul[@name="لون الخشب"]//li')
len(colors_bois)

colors_tissu = driver.find_elements(By.XPATH, '//ul[@name="لون القماش"]//li')
len(colors_tissu)



for bois in colors_bois:
    bois.click()
    print(bois.text)
    time.sleep(2)
    for tissu in colors_tissu:
        tissu.click()
        time.sleep(2)

In [69]:

def scrape_data(url1: str, bois: str, tissu: str):
    cat1 = url1['cat1']
    cat2 = url1['cat2']
    cat3 = url1['cat3']
    url = url1['url']
    logging.info(f'URL: {url}')
    time.sleep(1)
    r = driver.page_source
    soup = BeautifulSoup(r, "html.parser")
    name = soup.find('section').find('h1').text.strip()
    price = soup.find('del', {'class': 'product-formatted-price-old'}).text.replace('ر.س', '').strip()
    special_price = soup.find('h1', {'class': 'product-formatted-price'}).text.replace('ر.س', '').strip()
    sku = soup.find('div', {'class': 'product-sku'}).text.strip()
    try:
        description = soup.find('div', {'class': 'description-paragrah'}).find('p').text.strip()
    except:
        description = soup.find('div', {'class': 'description-paragrah'}).text.strip()
    images = soup.find('div', {'class': 'product-images-carousel-thumbs'}).find_all('img')
    list_images = [img['src'].replace('-thumbnail-370x370-70.jpg', '-thumbnail-770x770-70.jpg') for img in images]
    base_image = list_images[0]
    add_images = ','.join(list_images[1:])
    data = {
        
        'sku': sku,
        'name': name,
        'price': price,
        'link_url': url,
        'special_price': special_price,
        'description': description,
        'color_bois': bois,
        'color_tissu': tissu,
        'base_image': base_image,
        'add_images': add_images,
        'cat1': cat1,
        'cat2': cat2,
        'cat3': cat3,
    }
    return data

In [70]:
def scraping_data(url1, driver):
    cat1 = url1['cat1']
    cat2 = url1['cat2']
    cat3 = url1['cat3']
    url = url1['url']
    driver.get(url)
    colors_bois = driver.find_elements(By.XPATH, '//ul[@name="لون الخشب"]//li')
    len(colors_bois)
    colors_tissu = driver.find_elements(By.XPATH, '//ul[@name="لون القماش"]//li')
    len(colors_tissu)
    for bois in colors_bois:
        bois.click()
        print(bois.text)
        time.sleep(1)
        for tissu in colors_tissu:
            tissu.click()
            print(tissu.text)
            time.sleep(1)
            data = scrape_data(url1, bois.text, tissu.text)
        

In [ ]:
df = pd.read_excel('astore_product_model.xlsx')
for i, url1 in enumerate(list_urls[34: ]):
    print(f'Count: {i}')
    print(url1['url'])
    cat1 = url1['cat1']
    cat2 = url1['cat2']
    cat3 = url1['cat3']
    url = url1['url']
    driver.get(url)
    colors_bois = driver.find_elements(By.XPATH, '//ul[@name="لون الخشب"]//li')
    len(colors_bois)
    colors_tissu = driver.find_elements(By.XPATH, '//ul[@name="لون القماش"]//li')
    len(colors_tissu)
    for bois in colors_bois:
        bois.click()
        print(bois.text)
        time.sleep(1)
        for tissu in colors_tissu:
            tissu.click()
            print(tissu.text)
            time.sleep(1)
            data = scrape_data(url1, bois.text, tissu.text)
    
            df1 = pd.DataFrame([data])
            df = pd.concat([df, df1], ignore_index=True)
            df.to_excel('astore_product_update1.xlsx')

In [66]:
r = driver.page_source
soup = BeautifulSoup(r, "html.parser")

In [67]:
description = soup.find('div', {'class': 'description-paragrah'}).text.strip()
description

'طقم جلسة خشب\xa0نوع الخشب نمساوي معالج للماء والرطوبه\xa0عدد الاشخاص ٨\xa0عدد القطع ٥\xa0٢ كراسي ثلاثيه مقاس ٢٠٠ سم٢ كراسي مفرده مقاس ٧٥ سم١ طاولة كبيره مقاس ٨٠*٨٠ سمعمق الجلسه ٧٠سم'

In [72]:
df1 = pd.read_excel('astore_product_update.xlsx')

In [73]:
df = pd.concat([df, df1], ignore_index=True)

In [ ]:
df.to_excel('astore_product_update2.xlsx')

In [13]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
cookies = {'session': '134-8225175-0355220'}
r = requests.get(url, headers=headers, cookies=cookies)
soup = BeautifulSoup(r.content, "html.parser")
time.sleep(1)


In [19]:
name = soup.find('section').find('h1').text.strip()
price = soup.find('del', {'class': 'product-formatted-price-old'}).text.replace('ر.س', '').strip()
special_price = soup.find('h1', {'class': 'product-formatted-price'}).text.replace('ر.س', '').strip()
sku = soup.find('div', {'class': 'product-sku'}).text.strip()
description = soup.find('div', {'class': 'description-paragrah'}).find('p').text.strip()
images = soup.find('div', {'class': 'product-images-carousel-thumbs'}).find_all('img')
len(images)

list_images = [img['src'].replace('-thumbnail-370x370-70.jpg', '-thumbnail-770x770-70.jpg') for img in images]
base_image = list_images[0]
add_images = ','.join(list_images[1:])
description

'طاولة طعام عدد الكراسي (6) ( نوع الخشب زان روماني - كراسي قطيفة - مطعم بالإستيل - تصميم تركي )'

In [24]:
images = soup.find('div', {'class': 'product-images-carousel-thumbs'}).find_all('img')
len(images)

list_images = [img['src'].replace('-thumbnail-370x370-70.jpg', '-thumbnail-770x770-70.jpg') for img in images]
base_image = list_images[0]
add_images = ','.join(list_images[1:])

['https://media.zid.store/thumbs/619ecb89-65c1-4512-a8db-867a088be3b3/cb054958-8e0c-4960-9361-ef4ce02210f5-thumbnail-770x770-70.jpg',
 'https://media.zid.store/thumbs/619ecb89-65c1-4512-a8db-867a088be3b3/39115028-e0bb-4722-bfb2-f974883cdedd-thumbnail-770x770-70.jpg',
 'https://media.zid.store/thumbs/619ecb89-65c1-4512-a8db-867a088be3b3/3d371557-8796-4943-a714-2f67edf69538-thumbnail-770x770-70.jpg',
 'https://media.zid.store/thumbs/619ecb89-65c1-4512-a8db-867a088be3b3/e317ec16-87c8-4559-8636-3afdda877529-thumbnail-770x770-70.jpg',
 'https://media.zid.store/thumbs/619ecb89-65c1-4512-a8db-867a088be3b3/5f00c38b-5f38-4f80-a9df-f5a16ec44096-thumbnail-770x770-70.jpg',
 'https://media.zid.store/thumbs/619ecb89-65c1-4512-a8db-867a088be3b3/d6155428-0b3c-42bb-9829-5c3f4369f4d3-thumbnail-770x770-70.jpg']